## LeNet

这是一个1998年由Yann LeCun创建的卷积神经网络架构，目前它已经被广泛用于手写数字识别中。我们将在本文件中实现它。

它的神经网络架构如下：
输入层(32\*32)，卷积层(28\*28)，平均池化层(14\*14)，卷积层(10\*10)，平均池化层(5\*5)，卷积层(1*1)，全连接层，全连接层。

### 1. 导入必要模块

In [1]:
import time as time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

### 2. 引入数据集

在这里，我们直接使用tensorflow中自带的数据集。

In [2]:
# These variables are all in type of numpy.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 3. 数据预处理

将28\*28的图片填充到32\*32的规模，以便于进行输入。将图片变为3维，以便于神经网络的训练。同时，将分类变为one-hot编码，以便于后续在神经网络训练中可以使用categorical cross-entropy损失函数。

In [3]:
train_images_32 = np.zeros((60000, 32, 32), dtype=train_images.dtype)
test_images_32 = np.zeros((10000, 32, 32), dtype=test_images.dtype)

start_row = (32 - 28) // 2
start_col = (32 - 28) // 2
for i in range(60000):
  train_images_32[i][start_row:start_row+28, start_col:start_col+28] = train_images[i]
for i in range(10000):
  test_images_32[i][start_row:start_row+28, start_col:start_col+28] = test_images[i]

train_images_32 = train_images_32.reshape((60000, 32, 32, 1)).astype('float32') / 255
test_images_32 = test_images_32.reshape((10000, 32, 32, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_images_32.shape)
print(test_images_32.shape)
print(train_labels.shape)
print(test_labels.shape)

(60000, 32, 32, 1)
(10000, 32, 32, 1)
(60000, 10)
(10000, 10)


### 4. 搭建神经网络

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.AveragePooling2D((2,2)))
model.add(layers.Conv2D(16, (5, 5), activation='relu'))
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.Conv2D(120, (5, 5), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dense(10, activation='sigmoid'))

### 5. 训练神经网络

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
start_time = time.time()
model.fit(train_images_32, train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time = time.time()
print("Training Time:", end_time - start_time, "seconds")

Epoch 1/5
750/750 [==============================] - 15s 6ms/step - loss: 0.3157 - accuracy: 0.9084 - val_loss: 0.1237 - val_accuracy: 0.9632
Epoch 2/5
750/750 [==============================] - 4s 5ms/step - loss: 0.0988 - accuracy: 0.9706 - val_loss: 0.0899 - val_accuracy: 0.9754
Epoch 3/5
750/750 [==============================] - 3s 5ms/step - loss: 0.0736 - accuracy: 0.9774 - val_loss: 0.0634 - val_accuracy: 0.9803
Epoch 4/5
750/750 [==============================] - 5s 6ms/step - loss: 0.0585 - accuracy: 0.9815 - val_loss: 0.0690 - val_accuracy: 0.9798
Epoch 5/5
750/750 [==============================] - 4s 5ms/step - loss: 0.0482 - accuracy: 0.9852 - val_loss: 0.0612 - val_accuracy: 0.9807
Training Time: 30.530242204666138 seconds


### 6. 观察测试集的效果

In [ ]:
test_loss, test_acc = model.evaluate(test_images_32, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 1s 3ms/step - loss: 0.0356 - accuracy: 0.9888
Test accuracy: 0.9887999892234802


容易发现，最后得到的效果非常好，我们搭建的LeNet比较成功。